In [1]:
import bridgestan as bs
from pathlib import Path
import os
from utils import *
import json
import pickle
import arviz as az
import numpy as np
from tqdm import tqdm
pkey=1
n_repeat=100
evaluating_model='DirichletSymmetric'
data_file = lambda *, pkey, n_repeat, transform, evaluating_model: f"/mnt/home/mjhajaria/ceph/sampling_results/simplex/{transform}/{evaluating_model}/samples_{pkey}_{n_repeat}.nc"
stan_filename = lambda *, transform, evaluating_model: f"stan_models/{transform}_{evaluating_model}.stan"
output_filename = lambda *, transform, evaluating_model, x, pkey: f'output/{transform}/{evaluating_model}/{x}_{pkey}.npy'

/mnt/home/mjhajaria/miniconda3/envs/stan/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for transform in tqdm(list_transforms()):
    stan_file=stan_filename(transform=transform, evaluating_model=evaluating_model)
    with open(stan_file, 'w') as f:
        f.write(f'#include target_densities/DirichletSymmetric.stan{os.linesep}#include transforms/{transform}.stan{os.linesep}')
        f.close()
    data= f'utils/dirichletsymmetricdata/data_{pkey}.json'
    bsmodel = bs.StanModel.from_stan_file(stan_file, data, 
                                          make_args=["BRIDGESTAN_AD_HESSIAN=true"],
                                          stanc_args=[f"--include-paths='/mnt/home/mjhajaria/simplex-transforms/'"])
    n=bsmodel.param_unc_num()
    idata_file = data_file(pkey=pkey, n_repeat=n_repeat, transform=transform, evaluating_model=evaluating_model)
    y = az.extract( az.from_netcdf(idata_file)).y.T.values[:1000]
    condno=[]
    for row in tqdm(y):
#         theta = bsmodel.param_unconstrain(np.ascontiguousarray(row))
        lp, grad, hess = bsmodel.log_density_hessian(row)
        condno.append(np.linalg.cond(hess, 2))
    print(len(condno))
    np.save(output_filename(transform=transform, 
                            evaluating_model=evaluating_model, pkey=pkey, x="cn"), np.asarray(condno))

  0%|                                                                                                                                                          | 0/7 [00:16<?, ?it/s]


RuntimeError: Command make BRIDGESTAN_AD_HESSIAN=true STANCFLAGS=--include-paths=. --include-paths='/mnt/home/mjhajaria/simplex-transforms/' /mnt/home/mjhajaria/simplex-transforms/stan_models/Stickbreaking_DirichletSymmetric_model.so failed with code 2.
stdout:

--- Translating Stan model to C++ code ---
./bin/stanc --include-paths=. --include-paths='/mnt/home/mjhajaria/simplex-transforms/' --o=/mnt/home/mjhajaria/simplex-transforms/stan_models/Stickbreaking_DirichletSymmetric.hpp /mnt/home/mjhajaria/simplex-transforms/stan_models/Stickbreaking_DirichletSymmetric.stan

--- Compiling C++ code ---
g++ -fvisibility-inlines-hidden -std=c++17 -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /mnt/home/mjhajaria/miniconda3/envs/stan/include -std=c++1y -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I ./stan/lib/stan_math/lib/tbb_2020.3/include    -O3 -I ./stan/src -I ./stan/lib/rapidjso

In [ ]:
cntransforms={}

for transform in tqdm(list_transforms()):
    cntransforms[transform] = np.load(output_filename(transform=transform, 
                            evaluating_model=evaluating_model, pkey=pkey, x="cn"))

In [ ]:
cntransforms['Stickbreaking']

In [ ]:
cntransforms['ALR']

In [ ]:
for transform in list_transforms():
    print(transform, np.ma.masked_invalid(cntransforms[transform]).mean())

In [ ]:
hess